In [2]:
import pandas as pd
import plotly_express as px
from hash_data import Hash_DataFrame as hd

# file path 
file_path = "Data/athlete_events.csv"

# load the data as a dataFrame 
df_data = pd.read_csv(file_path)

# encrypt the athletes names. TODO check why error message is not working.
df_data = hd.hash_Columns(df_data, ["Name"])

df_data


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,3a4eef48434c66b3f14ab0221f6762d0ef7c6135ab2790...,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,a6430cc6630934275dc6283f7e97e9625e6587cdddec7a...,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,9c198b205332c2c8e1542e0f9534b9e270780a41d978ec...,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,0a477bb1c5ad39716f9c775e54d18d16aa8b37ada55548...,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,5b7be356aa28178096dc6747f0b8e4e393eaceb5f95310...,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,07f491e055f56a7a7be133c9cf6563c253d3aa0d137630...,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN
271112,135570,2f748d07ca91a5a1a6871b8f05cdd4d69f194615a82ca5...,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
271113,135570,2f748d07ca91a5a1a6871b8f05cdd4d69f194615a82ca5...,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
271114,135571,e7fd567d163933f9b76a0094f18afbf8e8291b179c5487...,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN


In [13]:
# de sporter landet fått flest medaljer i

# get all swe data from dataframe
df_data_sweden = df_data.loc[df_data["Team"] == "Sweden"]

# groupby sport and count medals 
df_sport_medal = df_data_sweden[["Sport", "Medal"]].groupby("Sport").count().reset_index()

#NOTE i lag sport är det en medalj per lag medlem.
fig = px.bar(df_sport_medal, x= "Sport", y= "Medal")
fig.show()